Finetune with MBart 

In [1]:
!nvidia-smi

Fri Jan 13 17:10:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:4F:00.0 Off |                    0 |
| 30%   26C    P8    22W / 300W |      0MiB / 45631MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A6000    Off  | 00000000:52:00.0 Off |                    0 |
| 30%   

In [2]:
%env CUDA_VISIBLE_DEVICES=0
%env TOKENIZERS_PARALLELISM=false

env: CUDA_VISIBLE_DEVICES=0
env: TOKENIZERS_PARALLELISM=false


In [3]:
from datasets import load_dataset, concatenate_datasets
import transformers
from transformers import MBartModel, MBartTokenizer, MBartConfig, pipeline, Trainer, TrainingArguments, MBartForConditionalGeneration, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import evaluate
import numpy as np
import torch
from torch import tensor 
import os

Dataload using Huggingface 

In [20]:
file_path = "/home/sumire/discourse_context_mt/data/BSD-master/"
data_files = {"train": f"{file_path}train.json", "validation": f"{file_path}dev.json", "test": f"{file_path}test.json"}
dataset = load_dataset("json", data_files=data_files)
dataset

Using custom data configuration default-d1314a26020b1cab
Found cached dataset json (/home/sumire/.cache/huggingface/datasets/json/default-d1314a26020b1cab/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tag', 'title', 'original_language', 'conversation'],
        num_rows: 670
    })
    validation: Dataset({
        features: ['id', 'tag', 'title', 'original_language', 'conversation'],
        num_rows: 69
    })
    test: Dataset({
        features: ['id', 'tag', 'title', 'original_language', 'conversation'],
        num_rows: 69
    })
})

In [6]:
# define train inputs and targets
after_context = "</s>"
context_size = 1

inputs = [sent['en_sentence'] for doc in dataset["train"]["conversation"] for sent in doc]
targets = [sent['ja_sentence'] for doc in dataset["train"]["conversation"] for sent in doc]

print (inputs[:5], targets[:5])

['Hi this is the systems development department of Company K.', 'My name is Takaichi from Company H.', 'Thank you as always.', 'Thank you as always as well.', 'Is Inada-san there?'] ['はい、K社システム開発部です。', 'H社の高市と申します。', 'いつもお世話になっております。', 'こちらこそ、お世話になっております。', '稲田さんはいらっしゃいますか？']


In [17]:
# Context-aware inputs
# Input window size = 1
context_size = 1
new_inputs = []

for idx, input in enumerate(inputs):
    if idx-context_size < 0:
        #print (idx)
        new_input = f"<>{input}"
        #print (new_input)
        new_inputs.append(new_input)

    elif 0 <= idx-context_size:
        #print (idx)
        new_input = f"{inputs[idx-1]}</s>{input}"
        #print (new_input)
        new_inputs.append(new_input)

#print (new_inputs[0], new_inputs[-1])
print (len(new_inputs))
print (new_inputs[:5])

20000
['<>Hi this is the systems development department of Company K.', 'Hi this is the systems development department of Company K.</s>My name is Takaichi from Company H.', 'My name is Takaichi from Company H.</s>Thank you as always.', 'Thank you as always.</s>Thank you as always as well.', 'Thank you as always as well.</s>Is Inada-san there?']


Preprocess Using Huggingface

In [32]:

# facebook/mbart-large-50-many-to-many-mmt
model_checkpoint = "facebook/mbart-large-50-many-to-many-mmt"
configuration = MBartConfig()
tokenizer = MBartTokenizer.from_pretrained(model_checkpoint, src_lang="en_XX", tgt_lang="ja_XX")
model = MBartForConditionalGeneration(configuration).from_pretrained(model_checkpoint)

max_length = 128

def preprocess_function(data): # data should be splitted into train / dev / test internally
    inputs = [sent['en_sentence'] for doc in data["conversation"] for sent in doc]
    targets = [sent['ja_sentence'] for doc in data["conversation"] for sent in doc]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MBart50Tokenizer'. 
The class this function is called from is 'MBartTokenizer'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [28]:
import NotebookApp
NotebookApp.iopub_data_rate_limit=5000000

ModuleNotFoundError: No module named 'NotebookApp'

In [33]:
model_inputs = preprocess_function(dataset)
print ((model_inputs))

KeyError: 'conversation'

In [ ]:
# mbart-large-cc25
"""
model_checkpoint = "facebook/mbart-large-cc25"
configuration = MBartConfig()
tokenizer = MBartTokenizer.from_pretrained(model_checkpoint, src_lang="en_XX", tgt_lang="ja_XX")
model = MBartForConditionalGeneration(configuration).from_pretrained(model_checkpoint)

max_length = 128

def preprocess_function(data): # data should be splitted into train / dev / test internally
    inputs = [sent['en_sentence'] for doc in data["conversation"] for sent in doc]
    targets = [sent['ja_sentence'] for doc in data["conversation"] for sent in doc]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs
"""


'\nmodel_checkpoint = "facebook/mbart-large-cc25"\nconfiguration = MBartConfig()\ntokenizer = MBartTokenizer.from_pretrained(model_checkpoint, src_lang="en_XX", tgt_lang="ja_XX")\nmodel = MBartForConditionalGeneration(configuration).from_pretrained(model_checkpoint)\n\nmax_length = 128\n\ndef preprocess_function(data): # data should be splitted into train / dev / test internally\n    inputs = [sent[\'en_sentence\'] for doc in data["conversation"] for sent in doc]\n    targets = [sent[\'ja_sentence\'] for doc in data["conversation"] for sent in doc]\n    model_inputs = tokenizer(\n        inputs, text_target=targets, max_length=max_length, truncation=True\n    )\n    return model_inputs\n'

In [34]:
tokenized_datasets = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

Loading cached processed dataset at /home/sumire/.cache/huggingface/datasets/json/default-d1314a26020b1cab/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-c69a69d508e0c92a.arrow
Loading cached processed dataset at /home/sumire/.cache/huggingface/datasets/json/default-d1314a26020b1cab/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-bfb3dd6fb328072e.arrow
Loading cached processed dataset at /home/sumire/.cache/huggingface/datasets/json/default-d1314a26020b1cab/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-7fd9a99bf152e87f.arrow


DataCollator

In [16]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt") # not tf

Define Batch using Datacollator
###To Chryssa, decoder_input_ids now exists!####

In [17]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

Fine-tune using Trainer

In [18]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

Evaluation

In [19]:
metric = evaluate.load("sacrebleu")

In [20]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
"""
from torch import nn

def freeze_params(model: nn.Module):
    Set requires_grad=False for each of model.parameters()
    for par in model.parameters():
        par.requires_grad = False

#model = AutoModel.from_pretrained("facebook/bart-large")
enc_layers = model.get_encoder().layers
freeze_params(enc_layers)  # freeze layer 0
dropout = enc_layers[0].dropout   # return dropout value for layer 0
enc_layers[0].dropout = 0.5  # set dropout value for layer 0

"""

'\nfrom torch import nn\n\ndef freeze_params(model: nn.Module):\n    Set requires_grad=False for each of model.parameters()\n    for par in model.parameters():\n        par.requires_grad = False\n\n#model = AutoModel.from_pretrained("facebook/bart-large")\nenc_layers = model.get_encoder().layers\nfreeze_params(enc_layers)  # freeze layer 0\ndropout = enc_layers[0].dropout   # return dropout value for layer 0\nenc_layers[0].dropout = 0.5  # set dropout value for layer 0\n\n'

In [21]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"
#os.environ["CUDA_LAUNCH_BLOCKING"]="0"

training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,        
    #logging_dir='./logs',            
    num_train_epochs=3, #3             
    per_device_train_batch_size=1, #16  
    per_device_eval_batch_size=1,  #64 
    warmup_steps=500,                
    weight_decay=0.01,
    save_total_limit=3,
    predict_with_generate=True,
    report_to="all",
    fp16=True,
    #gradient_accumulation_steps=1000,
    #half_precision_backend="apex"
)

trainer = Seq2SeqTrainer(
    model=model,                         
    args=training_args,                  
    train_dataset=tokenized_datasets["train"],        
    eval_dataset=tokenized_datasets["validation"],            
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using cuda_amp half precision backend


In [22]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"
trainer.train()

/home/sumire/miniconda3/envs/mt2022_server/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 20000
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 60000
  Number of trainable parameters = 610879488


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# Training without using Trainer
##Doesn't work###

inputs = [sent['en_sentence'] for doc in dataset["train"]["conversation"] for sent in doc]
targets = [sent['ja_sentence'] for doc in dataset["train"]["conversation"] for sent in doc]

inputs = tokenizer(inputs, text_target=targets, return_tensors="pt", padding=True)
print (inputs)

model(**inputs)

Inference

In [ ]:
from transformers import pipeline
import protobuf

text = "Hi, this is Sumire. May I help you ?"
translator = pipeline("translation", model="./results/checkpoint-10000")
translator(text)